In [1]:
import Pkg
using Pkg

# Install required packages
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("StatsModels")
Pkg.add("GLM")
Pkg.add("Random")
Pkg.add("MLDataUtils")
Pkg.add("MLBase")
Pkg.add("FixedEffectModels")
Pkg.add("Lasso")
Pkg.add("MLJ")
Pkg.add("DecisionTree")
Pkg.add("RData")
Pkg.add("GLMNet")
Pkg.add("PrettyTables")
Pkg.add("MLJScikitLearnInterface")
Pkg.add("MLJFlux")
Pkg.add("Flux")

   Resolving package versions...
     Project No packages added to or removed from `C:\Users\ALBERTO TRELLES\.julia\environments\v1.12\Project.toml`
    Manifest No packages added to or removed from `C:\Users\ALBERTO TRELLES\.julia\environments\v1.12\Manifest.toml`
   Resolving package versions...
     Project No packages added to or removed from `C:\Users\ALBERTO TRELLES\.julia\environments\v1.12\Project.toml`
    Manifest No packages added to or removed from `C:\Users\ALBERTO TRELLES\.julia\environments\v1.12\Manifest.toml`
   Resolving package versions...
     Project No packages added to or removed from `C:\Users\ALBERTO TRELLES\.julia\environments\v1.12\Project.toml`
    Manifest No packages added to or removed from `C:\Users\ALBERTO TRELLES\.julia\environments\v1.12\Manifest.toml`
   Resolving package versions...
     Project No packages added to or removed from `C:\Users\ALBERTO TRELLES\.julia\environments\v1.12\Project.toml`
    Manifest No packages added to or removed from `C:

In [2]:
using CSV, DataFrames, StatsModels, GLM, Random, MLDataUtils, MLBase, FixedEffectModels, Lasso, MLJ, DecisionTree, RData, GLMNet, PrettyTables, MLJScikitLearnInterface, MLJFlux, Flux

In [16]:
using CSV, DataFrames, StatsModels, GLM, Random, RData, MLBase, MLJ, PrettyTables, FixedEffectModels
using MLDataUtils, FixedEffectModels, DecisionTree, Lasso, GLMNet, MLJScikitLearnInterface

# Debiased ML for Partially Linear Model in Julia

This is a simple implementation of Debiased Machine Learning for the Partially Linear Regression Model.

Reference: 

https://arxiv.org/abs/1608.00060


https://www.amazon.com/Business-Data-Science-Combining-Accelerate/dp/1260452778

The code is based on the book.

## Load data

In [3]:
url = "https://github.com/d2cml-ai/14.388_jl/raw/main/data/GrowthData.RData"
download(url, "data.RData")
rdata_read = RData.load("data.RData")
data = rdata_read["GrowthData"]
names(data)
println("Number of Rows : ", size(data)[1],"\n","Number of Columns : ", size(data)[2],) #rows and columns

Number of Rows : 90
Number of Columns : 63


In [4]:
data

Row,Outcome,intercept,gdpsh465,bmp1l,freeop,freetar,h65,hm65,hf65,p65,pm65,pf65,s65,sm65,sf65,fert65,mort65,lifee065,gpop1,fert1,mort1,invsh41,geetot1,geerec1,gde1,govwb1,govsh41,gvxdxe41,high65,highm65,highf65,highc65,highcm65,highcf65,human65,humanm65,humanf65,hyr65,hyrm65,hyrf65,no65,nom65,nof65,pinstab1,pop65,worker65,pop1565,pop6565,sec65,secm65,secf65,secc65,seccm65,seccf65,syr65,syrm65,syrf65,teapri65,teasec65,ex1,im1,xr65,tot1
,Float64,Int32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.0243358,1,6.59167,0.2837,0.153491,0.043888,0.007,0.013,0.001,0.29,0.37,0.21,0.04,0.06,0.02,6.67,0.16,3.69387,0.0203,6.68,0.165,0.11898,0.0195,0.0176,0.019,0.0931,0.1158,0.07877,0.12,0.23,0.01,0.09,0.18,0.01,0.301,0.568,0.043,0.004,0.008,0.0,89.46,79.98,98.61,0.0,12359,0.3469,0.4441,0.0275912,0.45,0.75,0.17,0.13,0.21,0.04,0.033,0.057,0.01,47.6,17.3,0.0729,0.0667,0.348,-0.014727
2,0.100473,1,6.82979,0.6141,0.313509,0.061827,0.019,0.032,0.007,0.91,1.0,0.65,0.16,0.23,0.09,6.97,0.145,3.93378,0.0185,7.114,0.154,0.12048,0.0556,0.0369,0.019,0.1589,0.156,0.09999,0.7,1.18,0.2,0.63,1.04,0.2,0.706,1.138,0.257,0.027,0.045,0.008,89.1,82.35,96.1,0.02325,4630,0.2703,0.4474,0.0356371,3.0,4.74,1.2,1.36,2.05,0.64,0.173,0.274,0.067,57.1,18.0,0.094,0.1438,0.525,0.00575
3,0.0670515,1,8.89508,0.0,0.204244,0.009186,0.26,0.325,0.201,1.0,1.0,1.0,0.56,0.62,0.51,3.11,0.024,4.27388,0.0188,3.662,0.027,0.23098,0.0465,0.0365,0.04,0.1442,0.1367,0.06,16.67,17.95,15.41,4.5,5.7,3.31,8.317,8.249,8.384,0.424,0.473,0.375,1.4,1.4,1.4,0.0,19678,0.3874,0.3175,0.0766846,36.74,33.5,39.95,15.68,13.19,18.14,2.573,2.478,2.667,26.5,20.7,0.1741,0.175,1.082,-0.01004
4,0.0640892,1,7.56528,0.1997,0.248714,0.03627,0.061,0.07,0.051,1.0,1.0,1.0,0.24,0.22,0.31,6.26,0.072,4.16821,0.0345,6.83,0.085,0.12928,0.0375,0.035,0.011,0.1165,0.2018,0.15616,3.1,3.4,2.8,2.11,2.28,1.95,3.833,3.86,3.807,0.104,0.114,0.095,20.6,20.6,20.6,0.0,1482,0.3011,0.4671,0.0310391,7.6,7.5,7.7,2.76,2.89,2.63,0.438,0.453,0.424,27.8,22.7,0.1265,0.1496,6.625,-0.002195
5,0.0279295,1,7.1624,0.174,0.299252,0.037367,0.017,0.027,0.007,0.82,0.85,0.81,0.17,0.15,0.13,6.71,0.12,3.9982,0.031,6.816,0.131,0.07932,0.0257,0.0224,0.012,0.0971,0.169,0.13427,0.67,0.98,0.36,0.45,0.66,0.25,1.9,2.084,1.72,0.022,0.033,0.012,58.73,55.56,61.82,0.2,3006,0.3314,0.4561,0.0262808,5.07,5.37,4.78,2.17,2.23,2.11,0.257,0.287,0.229,34.5,17.6,0.1211,0.1308,2.5,0.003283
6,0.0464074,1,7.21891,0.0,0.258865,0.02088,0.023,0.038,0.006,0.5,0.55,0.5,0.08,0.1,0.07,6.7,0.112,3.88978,0.0303,6.83,0.119,0.07608,0.0151,0.0156,0.009,0.0713,0.0734,0.04899,0.7,1.09,0.3,0.48,0.74,0.21,1.426,1.622,1.227,0.024,0.037,0.01,69.2,64.27,74.2,0.4242,4568,0.3105,0.4599,0.0273643,3.5,3.6,3.4,1.02,0.59,1.46,0.16,0.174,0.146,34.3,8.1,0.0634,0.0762,1.0,-0.001747
7,0.0673323,1,7.8536,0.0,0.182525,0.014385,0.039,0.063,0.014,0.92,0.94,0.92,0.17,0.21,0.12,6.72,0.082,4.08766,0.032,6.744,0.087,0.17044,0.0139,0.0134,0.007,0.0615,0.0675,0.0471,1.85,3.11,0.66,1.09,1.74,0.46,2.789,3.426,2.182,0.059,0.097,0.022,42.07,34.56,49.23,0.0,44752,0.2822,0.4612,0.0334287,5.28,6.64,3.98,2.42,3.28,1.59,0.342,0.484,0.207,46.6,14.7,0.0342,0.0428,12.499,0.009092
8,0.0209777,1,7.70391,0.2776,0.215275,0.029713,0.024,0.035,0.013,0.69,0.69,0.69,0.14,0.14,0.13,7.19,0.121,3.91999,0.0268,7.302,0.131,0.13006,0.0173,0.0165,0.018,0.0796,0.1137,0.07942,0.6,1.13,0.1,0.41,0.77,0.07,2.148,2.21,2.09,0.02,0.038,0.003,49.6,49.6,49.6,0.0,1750,0.2984,0.4812,0.0234286,4.3,4.83,3.8,1.25,0.85,1.63,0.184,0.219,0.152,34.0,16.1,0.0864,0.0931,7.0,0.01163
9,0.0335512,1,9.06346,0.0,0.109614,0.002171,0.

In [5]:
y = data[!,1]
# y= reshape(y, (length(y),1))
d = data[!,3]
# d= reshape(d, (length(y),1))
x = data[!,4:end];
# x = Matrix(x);

In [6]:
scitype(x)

Table{Union{AbstractVector{ScientificTypesBase.Continuous}, AbstractVector{Count}}}

In [7]:
coerce!(x, Count => MLJ.Continuous);

## Naive OLS that uses all features w/o cross-fitting

In [8]:
println("\n length of y is \n", size(y,1) )
println("\n num features x is \n", size(x,2 ) )

# Naive OLS
print( "\n Naive OLS that uses all features w/o cross-fitting \n" )
fm = term(:Outcome) ~ term(:gdpsh465) +sum(term.(Symbol.(names(data[:,4:size(data,2)]))));
lres = reg(data, fm);
first(DataFrame(GLM.coeftable(lres)))


 length of y is 
90

 num features x is 
60

 Naive OLS that uses all features w/o cross-fitting 


Row,Name,Estimate,Std. Error,t-stat,Pr(>|t|),Lower 95%,Upper 95%
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,gdpsh465,-0.00937799,0.0298877,-0.313774,0.756104,-0.0707025,0.0519466


## DML algorithm

Here we perform estimation and inference of predictive coefficient $\alpha$ in the partially linear statistical model,
$$
Y = D\alpha + g(X) + U, \quad E (U | D, X) = 0.
$$
For $\tilde Y = Y- E(Y|X)$ and $\tilde D= D- E(D|X)$, we can write
$$
\tilde Y = \alpha \tilde D + U, \quad E (U |\tilde D) =0.
$$
Parameter $\alpha$ is then estimated using cross-fitting approach to obtain the residuals $\tilde D$ and $\tilde Y$.
The algorithm comsumes $Y, D, X$, and machine learning methods for learning the residuals $\tilde Y$ and $\tilde D$, where
the residuals are obtained by cross-validation (cross-fitting).

The statistical parameter $\alpha$ has a causal interpretation of being the effect of $D$ on $Y$ in the causal DAG $$ D\to Y, \quad X\to (D,Y)$$ or the counterfactual outcome model with conditionally exogenous (conditionally random) assignment of treatment $D$ given $X$:
$$
Y(d) = d\alpha + g(X) + U(d),\quad  U(d) \text{ indep } D |X, \quad Y = Y(D), \quad U = U(D).
$$


In [9]:
function training_sample_append(cv_split, test_sample_index)
        training_indices = []
        for vector in cv_split[Not(test_sample_index)]
                training_indices = [training_indices; vector]
        end
        return training_indices, cv_split[test_sample_index]
end

function dml(x, d, y, dreg, yreg, nfold)
        n = length(y)
        cv = [partition(eachindex(y), fill(1/nfold, nfold-1)..., shuffle = true, rng = 1234)...]
        machine_y = machine(yreg, x, y, scitype_check_level=0)
        machine_d = machine(dreg, x, d, scitype_check_level=0)
        y_hat = zeros(n)
        d_hat = zeros(n)

        for fold in 1:nfold
                training_fold, test_fold = training_sample_append(cv, fold)
                y_hat[test_fold] = MLJ.predict(MLJ.fit!(machine_y, rows = training_fold), x[test_fold, :])
                d_hat[test_fold] = MLJ.predict(MLJ.fit!(machine_d, rows = training_fold), x[test_fold, :])
        end

        resy = y .- y_hat
        resd = reshape(d .- d_hat, (n, 1))
        estimate = lm(resd, resy)
        coef_est = GLM.coef(estimate)[1]
        se = GLM.coeftable(estimate).cols[2][1]
        println(" coef (se) = ", coef_est ,"(",se,")")
        return coef_est, se, resy, resd;
end

function summarize(point, stderr, resy, resd, name)
        return DataFrame(
                model = [name],
                estimate = [point], stderr = [stderr], 
                rmse_y = [sqrt(mean(resy .^ 2))], 
                rmse_d = [sqrt(mean(resd .^ 2))]
        )
end

summarize (generic function with 1 method)

In [44]:
LinearRegressor = @load LinearRegressor pkg=MLJScikitLearnInterface verbosity=0
dreg = Standardizer() |> LinearRegressor()
yreg = Standardizer() |> LinearRegressor()
result_ols = dml(x, d, y, dreg, yreg, 10)
table_ols = summarize(result_ols..., "OLS")

LassoCVRegressor = @load LassoCVRegressor pkg=MLJScikitLearnInterface verbosity=0
dreg = Standardizer() |> LassoCVRegressor(max_iter=200000)
yreg = Standardizer() |> LassoCVRegressor(max_iter=200000)
results_lasso = dml(x, d, y, dreg, yreg, 10)
table_lasso = summarize(results_lasso..., "LassoCV")

RandomForestRegressor = @load RandomForestRegressor pkg=MLJScikitLearnInterface verbosity=0
dreg = RandomForestRegressor()
yreg = RandomForestRegressor()
results_rf = dml(x, d, y, dreg, yreg, 10)
table_rf = summarize(results_rf..., "RF")

dreg = Standardizer() |> LassoCVRegressor(max_iter=200000)
results_mix = dml(x, d, y, dreg , yreg, 10)
table_mix = summarize(results_mix..., "RF/LassoCV")

NeuralNetworkRegressor = @load NeuralNetworkRegressor pkg=MLJFlux verbosity=0 
nn_model = NeuralNetworkRegressor(
    builder = MLJFlux.MLP(; hidden=(20,20), σ=relu), 
    epochs = 100,
    batch_size = 32,
    optimiser = Flux.ADAM(0.001),
    rng = 1234
)
dreg = Standardizer() |> nn_model
yreg = Standardizer() |> nn_model
results_nn = dml(x, d, y, dreg, yreg, 10)
table_nn = summarize(results_nn..., "NeuralNet")



[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).


 coef (se) = -0.002498994254548932(0.01854254771822516)


[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).


 coef (se) = -0.05500048033559874(0.014826076391788765)


[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(

...

[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[]

 coef (se) = -0.04976058673871041(0.013047699858061956)


[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:neural_network_regressor, …).
[ Info: MLJFlux: converting input data to Float32
Optimising neural net: 100%[=========================] Time: 0:00:00
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:neural_network_regressor, …).
[ Info: MLJFlux: converting input data to Float32
Optimising neural net: 100%[=========================] Time: 0:00:00
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:neural_network_regressor, …).
[ Info: MLJFlux: converting input data to Float32
Optimising neural net: 100%[=========================] Time: 0:00:00
[ Info: Training ma

 coef (se) = 0.04481648198984531(0.014483980010207394)


Row,model,estimate,stderr,rmse_y,rmse_d
,String,Float64,Float64,Float64,Float64
1,NeuralNet,0.0448165,0.014484,0.237082,1.64866


In [45]:
pretty_table([table_ols; table_lasso; table_rf; table_mix; table_nn])

┌────────────┬─────────────┬───────────┬───────────┬──────────┐
│      model │    estimate │    stderr │    rmse_y │   rmse_d │
│     String │     Float64 │   Float64 │   Float64 │  Float64 │
├────────────┼─────────────┼───────────┼───────────┼──────────┤
│        OLS │ -0.00249899 │ 0.0185425 │ 0.0780211 │ 0.445967 │
│    LassoCV │  -0.0550005 │ 0.0148261 │ 0.0541261 │ 0.360134 │
│         RF │  -0.0434742 │ 0.0118976 │ 0.0465705 │ 0.386905 │
│ RF/LassoCV │  -0.0497606 │ 0.0130477 │ 0.0478148 │ 0.360134 │
│  NeuralNet │   0.0448165 │  0.014484 │  0.237082 │  1.64866 │
└────────────┴─────────────┴───────────┴───────────┴──────────┘


In [46]:
vcat(table_ols, table_lasso, table_rf, table_mix, table_nn)

Row,model,estimate,stderr,rmse_y,rmse_d
,String,Float64,Float64,Float64,Float64
1,OLS,-0.00249899,0.0185425,0.0780211,0.445967
2,LassoCV,-0.0550005,0.0148261,0.0541261,0.360134
3,RF,-0.0434742,0.0118976,0.0465705,0.386905
4,RF/LassoCV,-0.0497606,0.0130477,0.0478148,0.360134
5,NeuralNet,0.0448165,0.014484,0.237082,1.64866
